# **Monitoreo de Modelos Desplegados en Azure Machine Learning**



### **1. Introducción al Monitoreo de Modelos**
El monitoreo de modelos es un proceso crítico para garantizar que los modelos en producción continúen funcionando correctamente a lo largo del tiempo. Los cambios en los datos o en el entorno pueden afectar el rendimiento del modelo, lo que puede llevar a predicciones incorrectas.

#### **1.1. ¿Por qué es importante el monitoreo?**
- **Detección de problemas:** Identificar rápidamente problemas como la deriva de datos o la degradación del rendimiento.
- **Mantenimiento continuo:** Garantizar que los modelos sigan siendo relevantes y precisos.
- **Toma de decisiones:** Proporcionar información para decidir si un modelo necesita ser reentrenado o actualizado.

---

### **2. Opciones de Monitoreo en Azure ML**
Azure ML ofrece múltiples herramientas y características para monitorear modelos desplegados. A continuación, se describen las principales opciones:

#### **2.1. Métricas de Rendimiento en Tiempo Real**
- **Descripción:**  
  - Azure ML permite registrar métricas clave en tiempo real, como precisión, recall, F1-score (para clasificación) o RMSE, MAE, R² (para regresión).
- **Cuándo usarlo:**  
  - Para evaluar continuamente el rendimiento del modelo en producción.
- **Cómo configurarlo:**  
  - Usa el SDK de Azure ML para registrar métricas durante la inferencia.
  ```python
  from azureml.core import Run

  run = Run.get_context()
  run.log("accuracy", accuracy_score(y_true, y_pred))
  run.log("precision", precision_score(y_true, y_pred))
  ```

---

#### **2.2. Detección de Deriva de Datos (Data Drift)**
- **Descripción:**  
  - La deriva de datos ocurre cuando las distribuciones de los datos de entrada cambian significativamente en comparación con los datos de entrenamiento. Esto puede afectar negativamente el rendimiento del modelo.
- **Cuándo usarlo:**  
  - Para identificar cambios en los patrones de datos que puedan invalidar las predicciones del modelo.
- **Cómo configurarlo:**  
  - Usa el módulo `DataDriftDetector` para monitorear la deriva de datos.
  ```python
  from azureml.datadrift import DataDriftDetector

  monitor = DataDriftDetector.create_from_model(
      workspace=ws,
      model_id=model.id,
      frequency="Day",  # Frecuencia de monitoreo
      feature_list=["feature1", "feature2"]  # Características a monitorear
  )
  monitor.run()
  ```

---

#### **2.3. Alertas y Notificaciones Automáticas**
- **Descripción:**  
  - Configura alertas para recibir notificaciones cuando el rendimiento del modelo caiga por debajo de un umbral o cuando se detecte deriva de datos.
- **Cuándo usarlo:**  
  - Para actuar rápidamente ante problemas críticos.
- **Cómo configurarlo:**  
  - Usa el módulo `AlertConfiguration` para configurar alertas.
  ```python
  from azureml.monitoring import AlertConfiguration

  alert_config = AlertConfiguration(emails=["admin@example.com"])
  monitor.update(alert_configuration=alert_config)
  ```

---

#### **2.4. Registros de Solicitudes y Respuestas**
- **Descripción:**  
  - Azure ML registra automáticamente las solicitudes y respuestas del modelo desplegado. Estos registros son útiles para depurar problemas y analizar el comportamiento del modelo.
- **Cuándo usarlo:**  
  - Para investigar errores o anomalías en las predicciones.
- **Cómo acceder a los registros:**  
  - Usa el SDK o el portal de Azure para acceder a los registros.
  ```python
  logs = service.get_logs()
  print(logs)
  ```

---

### **3. Ejemplos Prácticos**

#### **3.1. Monitoreo del Rendimiento del Modelo**
Supongamos que has desplegado un modelo de clasificación. Puedes monitorear su precisión y recall en tiempo real:
```python
from sklearn.metrics import accuracy_score, precision_score
from azureml.core import Run

# Simular datos reales
y_true = [0, 1, 1, 0]
y_pred = [0, 1, 0, 0]

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

# Registrar métricas en Azure ML
run = Run.get_context()
run.log("accuracy", accuracy)
run.log("precision", precision)
```

---

#### **3.2. Detección de Deriva de Datos**
Configura un monitor para detectar cambios en las características del modelo:
```python
from azureml.datadrift import DataDriftDetector

# Crear un detector de deriva de datos
monitor = DataDriftDetector.create_from_model(
    workspace=ws,
    model_id=model.id,
    frequency="Day",
    feature_list=["age", "income"]
)

# Ejecutar el monitor
monitor.run()
```

---

#### **3.3. Configuración de Alertas**
Configura alertas para recibir notificaciones cuando el modelo experimente problemas:
```python
from azureml.monitoring import AlertConfiguration

# Configurar alertas por correo electrónico
alert_config = AlertConfiguration(emails=["admin@example.com"])
monitor.update(alert_configuration=alert_config)
```

---

### **4. Ejercicio Práctico**

**Título:** Configurar monitoreo para un modelo de regresión desplegado en Azure ML.

#### **Pasos:**
1. **Despliegue del Modelo:**
   - Despliega un modelo de regresión en Azure ML utilizando ACI o AKS.
   ```python
   from azureml.core.webservice import AciWebservice
   from azureml.core.model import InferenceConfig

   inference_config = InferenceConfig(entry_script="score.py")
   deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

   service = Model.deploy(workspace=ws,
                          name="mi-servicio-regresion",
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=deployment_config)
   service.wait_for_deployment(show_output=True)
   ```

2. **Configuración de Métricas:**
   - Configura métricas de rendimiento (RMSE, MAE) para monitorear el modelo.
   ```python
   from sklearn.metrics import mean_squared_error
   from azureml.core import Run

   # Simular datos reales
   y_true = [3.0, 5.0, 7.0]
   y_pred = [2.8, 4.9, 6.8]

   # Calcular RMSE
   rmse = mean_squared_error(y_true, y_pred, squared=False)

   # Registrar métrica
   run = Run.get_context()
   run.log("rmse", rmse)
   ```

3. **Detección de Deriva de Datos:**
   - Configura un monitor para detectar cambios en las características del modelo.
   ```python
   from azureml.datadrift import DataDriftDetector

   monitor = DataDriftDetector.create_from_model(
       workspace=ws,
       model_id=model.id,
       frequency="Day",
       feature_list=["feature1", "feature2"]
   )
   monitor.run()
   ```

4. **Configuración de Alertas:**
   - Configura alertas para recibir notificaciones cuando el RMSE supere un umbral.
   ```python
   from azureml.monitoring import AlertConfiguration

   alert_config = AlertConfiguration(emails=["admin@example.com"])
   monitor.update(alert_configuration=alert_config)
   ```

---

### **Conclusión**
Este módulo proporciona una visión completa del monitoreo de modelos en Azure ML, desde la configuración de métricas de rendimiento hasta la detección de deriva de datos y la configuración de alertas. Los participantes estarán preparados para garantizar que sus modelos en producción sigan siendo precisos y confiables.

#### **Referencias Generales:**
- [Azure ML Monitoring Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-models)
- [Data Drift Detection in Azure ML](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-monitor-datasets)
- [Scikit-learn Metrics Documentation](https://scikit-learn.org/stable/modules/model_evaluation.html)